<a href="https://colab.research.google.com/github/gacerioni/Redis-VectorDB-LLM-RAG-Workshop/blob/main/02-Vector_Similarity_Search/02-Redis_VSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Vector Similarity Search com Redis

[Always-on demo by Anton](https://antonum-redis-vss-streamlit-streamlit-app-p4z5th.streamlit.app/)

![Redis](https://redis.io/wp-content/uploads/2024/04/Logotype.svg?auto=webp&quality=85,75&width=120)

Este notebook irá gerar embeddings para o VectorDB (Redis) usando um modelo pré-treinado gratuito: `sentence-transformers/all-MiniLM-L6-v2`.

In [1]:
# Instale o Redis client e tambem o Hugging Face sentence transformers, pois vamos gerar os vetores aqui mesmo
!pip install -q redis sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 16.3 MB/s eta 0:00:00


Instale o Redis Stack localmente, só pra garantir!\
*(vai que você não está no Redis Cloud... quero que participes do lab sem Toil)*

In [2]:
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes


deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


### Conectando com o Redis server

In [3]:
import os

REDIS_HOST = os.getenv("REDIS_HOST", "localhost")
REDIS_PORT = os.getenv("REDIS_PORT", "6379")
REDIS_PASSWORD = os.getenv("REDIS_PASSWORD", "")
#Replace values above with your own if using Redis Cloud instance
#REDIS_HOST="redis-18374.c253.us-central1-1.gce.cloud.redislabs.com"
#REDIS_PORT=18374
#REDIS_PASSWORD="1TNxTEdYRDgIDKM2gDfasupCADXXXX"

#shortcut for redis-cli $REDIS_CONN command
# If SSL is enabled on the endpoint add --tls
if REDIS_PASSWORD!="":
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT} -a {REDIS_PASSWORD} --no-auth-warning"
else:
  os.environ["REDIS_CONN"]=f"-h {REDIS_HOST} -p {REDIS_PORT}"

# If SSL is enabled on the endpoint, use rediss:// as the URL prefix
REDIS_URL = f"redis://:{REDIS_PASSWORD}@{REDIS_HOST}:{REDIS_PORT}"
INDEX_NAME = f"qna:idx"

# Test Redis connection
!redis-cli $REDIS_CONN PING

PONG


In [4]:
import redis
redis = redis.Redis(
  host=REDIS_HOST,
  port=REDIS_PORT,
  password=REDIS_PASSWORD)
redis.ping()

True

In [5]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from tqdm.auto import tqdm
from redis import Redis
from redis.commands.search.field import (
    NumericField,
    TagField,
    TextField,
    VectorField,
)
from redis.commands.search.indexDefinition import IndexDefinition, IndexType
from redis.commands.search.query import Query


tqdm.pandas()



### Embedding generation model

Aqui começa o samba. Vamos usar o `sentence-transformers/all-MiniLM-L6-v2` da HuggingFace. https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2 para gerar nosso embedding de vetores. Vai ser bacana, e vai abrir a mente sobre usos fora do mundo de Machine Learning... como buscas performáticas por features e aproximações semânticas.

Este modelo possui 384 dimensões, o que já é mais do que suficiente para a nossa demo, considerando os tokens e tudo mais.



In [6]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

#### Download 12k+ tweets

Vamos usar um material do Anton, nosso gênio de outra Region, para subir 12k de Tweets pro Redis. Baixe o csv aqui mesmo:

In [7]:
!wget https://raw.githubusercontent.com/antonum/Redis-VSS-Streamlit/main/Labelled_Tweets.csv

--2024-05-13 20:00:25--  https://raw.githubusercontent.com/antonum/Redis-VSS-Streamlit/main/Labelled_Tweets.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2486081 (2.4M) [text/plain]
Saving to: ‘Labelled_Tweets.csv’

Labelled_Tweets.csv 100%[===================>]   2.37M  --.-KB/s    in 0.01s   

2024-05-13 20:00:25 (233 MB/s) - ‘Labelled_Tweets.csv’ saved [2486081/2486081]



In [9]:
df = pd.read_csv('Labelled_Tweets.csv').drop(columns=['created_at','score'])
# opcional, pra gente segurar o reggae caso estejamos travados no cap de 30MB do Free Tier.
#df=df.head(3000) #trim dataframe to fit results into 30MB Redis database
df


,id,full_text
0,1,@KennyDegu very very little volume. With $10T ...
1,2,#ES_F achieved Target 2780 closing above 50% #...
2,3,RT @KimbleCharting: Silver/Gold indicator crea...
3,4,@Issaquahfunds Hedged our $MSFT position into ...
4,5,RT @zipillinois: 3 Surprisingly Controversial ...
...,...,...
12415,12587,RT @PeterLBrandt: $SPX $ES_F \r\nFollowing thi...
12416,12588,RT @vieiraUAE: Fearless Alex Vieira Calls Best...
12417,12589,$spy $spx $qqq $ndx #nyse going from poking th...
12418,12590,RT @DavidScottAdams: On watch tomorrow // Pt. ...


### Gerando os Embeddings (o Vetor em si, finalmente)

Vamos gerar o "vector embedding" para o nosso dataframe.
Este passo pode levar 2-4 minutinhos, pois a GPU aqui é de demo rsrs


Notem que eu optei por utilizar o campo `full_text` para gerar os embeddings. Entretanto, em breve você irá ficar craque em decidir o que usar para gerar essa "impressão digital" da Entidade. Um filme, por exemplo, poderia usar uma concatenação do sumário, gênero, resenhas, paleta de cores, etc.


In [ ]:
def text_to_embedding(text):
  return model.encode(text).astype(np.float32).tobytes()

#generate vector embeddings
df["text_embedding"] = df["full_text"].progress_apply(text_to_embedding)
df.head()

  0%|          | 0/12420 [00:00<?, ?it/s]

### Criando as Helper Functions que iremos utilizar

Aqui não tem segredo. Vamos desenhar as functions que irão nos apoiar nessa reta final. Já temos os embeddings, agora vamos guardar no Redis a entidade completa + embeddings, o que é uma boa prática.\

Aqui, vamos:
- Salvar os dados como HASH no Redis;
- Criar o RediSearch Index corretamente, explicando pro Redis o que é aquele embedding curioso que estamos adicionando no bolo.

In [ ]:
def load_dataframe(redis, df, key_prefix="tweet", id_column="id", pipe_size=100):
    records = df.to_dict(orient="records")
    pipe = redis.pipeline(transaction=False)
    i=1
    for record in tqdm(records):
        i=i+1
        key = f"{key_prefix}:{record[id_column]}"
        pipe.hset(key, mapping=record)
        if (i+1) % pipe_size == 0:
          res=pipe.execute()
    pipe.execute()

def create_redis_index(redis, idxname="tweet:idx"):
  try:
    redis.ft(idxname).dropindex()
  except:
    print("no index found")

  # Create an index
  indexDefinition = IndexDefinition(
      prefix=["tweet:"],
      index_type=IndexType.HASH,
  )

  redis.ft(idxname).create_index(
      (
          TextField("full_text", no_stem=False, sortable=False),
          VectorField("text_embedding", "HNSW", {  "TYPE": "FLOAT32",
                                                    "DIM": 384,
                                                    "DISTANCE_METRIC": "COSINE",
                                                  })
      ),
      definition=indexDefinition
  )



### Create index and load data to Redis

In [ ]:
# clear Redis database (optional)
redis.flushdb()

# create Index
create_redis_index(redis)

# load data from Dataframe to Redis HASH
load_dataframe(redis,df,key_prefix="tweet", pipe_size=100)


no index found


  0%|          | 0/12420 [00:00<?, ?it/s]

In [ ]:
#Check how the data is stored in Redis
!redis-cli $REDIS_CONN hgetall "tweet:1001"

1) "id"
2) "1001"
3) "text_embedding"
4) "\xd8\x1e\x03:\xdfb\xbd\xbcA\xb0\x94\xbc1K)<\xf4T\x94;%>\x11\xbd\xae3\r=\t\x98\x7f=\x99X\x88<uUR\xbd\xbc\xf6h\xbci\xb5\xd2\xbb\x15;\t\xbd\xf9m#<\xe7\xd4C=\x91\xc8\"\xbd\x1b\xa3-\xbd0^\t\xbdv\x00\xfd\xbc\xb6GE\xbdn\xadZ\xbc]\xae=\xbdH<\x99<\xf2b\xce\xbc\b`\xab<4DT\xbcmM\xd7\xbc\xdb\xf0\x82\xbb\x8e\xb0(=Gv\x8a\xbc\xd1\xa8M\xbdd,P=\bS\x86<\x8fj\xfa<??\xb5<\xd9[\xee\xbb\xcf\x8d\x90=\rW8=R\x81\x82\xbd\x96\xfb&<\xf2-:=Y\xe6\xe4\xbd\xae\x1d\x11\xbc\xd4\xb7\xac\xbd\xfbY<\xbbi\xc9\xcc\xbb\xf29b\xbcI\xe3\x90=0\x9f\xa7=\xef7\x0f=*\xbb\xbb\xbbP\xad\x17=\xb8\xaeE\xbd\xe2\x8e\x90=r\xd7\x02\xbd\xe0\x01\x9f\xbcv\xc3e;\a\xa5\xd8\xbc\x80{f<\x9bx\xaf\xbd\xf2\xc4\x89\xbcPo\x03=wB\x8e\xbc4\x91G;\xf8\xe6\x97\xba\xbd\xc1\x9a=\x12\xf7B\xbd\xb5\r\xca<_\xf2G\xbb\xbf\x0e\x86<\xb2r\x84<\xcd\xf7^\xbd5\xc9\xbe\xbd`K5\xbc\"\xe09\xbd\xc4\xd7==5y\xb3=\xad\x10\xb4\xbdr\xfc\x11\xbd\x11\x87O\xbc\xc9o<\xbd}\x87\x80\xbc\x1e\xa9\"\xbdV) \xbd\xe2\xfa\x85\xba\xc7y\x95=\xf2\xdf\xa6<\x03

## Query the database

[Alway-on Streamlit app](https://antonum-redis-vss-streamlit-streamlit-app-p4z5th.streamlit.app/)


Try queries like:
“Oil”, “Oil Reserves”, “Fossil fuels”

Lexical Full Text search quickly runs out of matches

Vector search continues to discover relevant tweets

In [ ]:
user_query="oil price"
# queries to try "oil reserve", "fossil fuels"

In [ ]:
#using Full Text Index
q = Query(user_query)\
  .return_fields("full_text")
res = redis.ft("tweet:idx").search(q)
if res.total==0:
  print("No matches found")
else:
  res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
  display(res_df)

,id,full_text
0,tweet:3220,The relative performance of TIPS has historica...
1,tweet:311,"Oil Prices Rise, Fall As Russia, Saudi Arabia ..."
2,tweet:1490,"RT @Benzinga: Oil Prices Rise, Fall As Russia,..."
3,tweet:1585,"RT @Benzinga: Oil Prices Rise, Fall As Russia,..."
4,tweet:1610,"Oil Prices Rise, Fall As Russia, Saudi Arabia ..."
5,tweet:7189,Do higher oil prices help the consumer and sma...
6,tweet:636,Told you Saudi Arabia will bend the knee @jimc...
7,tweet:5405,https://t.co/3IJBXa5wuf Historic oil price plu...
8,tweet:5406,Historic oil price plunge trashes sector's pro...
9,tweet:3865,Today's book recommendation goes for the winne...


In [ ]:
#using Vector Similarity Index
query_vector=text_to_embedding(user_query)
q = Query("*=>[KNN 10 @text_embedding $vector AS result_score]")\
                .return_fields("result_score","full_text")\
                .dialect(2)\
                .sort_by("result_score", True)
res = redis.ft("tweet:idx").search(q, query_params={"vector": query_vector})
#print(res)
res_df = pd.DataFrame([t.__dict__ for t in res.docs ]).drop(columns=["payload"])
res_df

,id,result_score,full_text
0,tweet:444,0.369450807571,Would you spend $2 more a gallon of gasoline i...
1,tweet:11529,0.371095597744,RT @tradingcrudeoil: Crude oil closed up $0.48...
2,tweet:5654,0.381934285164,..and oil still 25.74 LMAO &gt;&gt;&gt;NO DEM...
3,tweet:204,0.396132171154,Bad news for #oil. It’s going to between $10 ...
4,tweet:9189,0.409308671951,Oil erases gains for the day in fall to $25 ht...
5,tweet:7189,0.429816782475,Do higher oil prices help the consumer and sma...
6,tweet:9330,0.430081248283,The price of Texas intermediate oil (WTI) slum...
7,tweet:531,0.431391596794,#OIL Sentiment ($22.50)\r\n\r\nWhat’s next for...
8,tweet:178,0.441844820976,OH how bullish for #oil LOL\r\n\r\n#OOTT #Oi...
9,tweet:6867,0.442979931831,$DXY 99.55-0.57%&lt;==US Dollar lower #Fed $2....
